실습: 삼성전자 주가 데이터를 분석하는 어플리케이션을 function(tool)으로 수정하여, 다른 종목의 데이터를 정해진 내용대로(종목, 시작날짜, 종료날짜) 가져와서 답하는 어플리케이션을 작성하시오.

In [21]:
import os
import json
from openai import OpenAI

client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")
# print (client.api_key)

In [41]:
from pykrx import stock
from pykrx import bond
import json

def get_current_stock(compy_name,start_day='20250101' ,end_day='20251231', perido='d'):
    stock_info = stock.get_market_ohlcv(start_day, end_day, compy_name, perido)

    return json.dumps(stock_info.to_json())
print (get_current_stock('005930'))

"{\"\\uc2dc\\uac00\":{\"1735776000000\":52700,\"1735862400000\":52800,\"1736121600000\":54400,\"1736208000000\":56800,\"1736294400000\":54800,\"1736380800000\":57600,\"1736467200000\":56100,\"1736726400000\":54600,\"1736812800000\":54200,\"1736899200000\":54100,\"1736985600000\":54200,\"1737072000000\":53800,\"1737331200000\":53600,\"1737417600000\":53700,\"1737504000000\":53400,\"1737590400000\":53700,\"1737676800000\":53600,\"1738281600000\":52200,\"1738540800000\":51100,\"1738627200000\":51600,\"1738713600000\":53600,\"1738800000000\":53400,\"1738886400000\":53900,\"1739145600000\":53000,\"1739232000000\":55500,\"1739318400000\":55100,\"1739404800000\":56100,\"1739491200000\":56000,\"1739750400000\":56100,\"1739836800000\":56200,\"1739923200000\":57400,\"1740009600000\":59100,\"1740096000000\":58400,\"1740355200000\":57300,\"1740441600000\":56600,\"1740528000000\":57000,\"1740614400000\":56500,\"1740700800000\":55400,\"1741046400000\":53900,\"1741132800000\":55100,\"1741219200000\":

In [50]:
messages = [{"role": "user", "content": "오늘 005930 주가를 알려줘."}]
tools = [
    {
        "type": "function",
        "function":
        {
            "name": "get_current_stock",
            "description": "005930 2025년도 일단위 주가를 알려줘.",
            "parameters": {
                "type": "object",
                "properties": {
                    "compy_name": {
                        "compy_name": "string",
                        "description": "compy code 예) 005930 : 삼성전자",
                    },
                    "start_day": {"type": "string"},
                    "end_day": {"type": "string"},
                },
                "required": ["compy_name"],
            },
        }
    }
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    tool_choice="auto" if tools is not None else None
    )
response_message = response.choices[0].message
print(response_message)
print(response_message.tool_calls[0])
print(type(response_message.tool_calls[0]))
print(type(response_message.tool_calls[0].function))

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_TO4ARNyFp0GKrrlgCy9rQaxH', function=Function(arguments='{"compy_name":"005930"}', name='get_current_stock'), type='function')])
ChatCompletionMessageToolCall(id='call_TO4ARNyFp0GKrrlgCy9rQaxH', function=Function(arguments='{"compy_name":"005930"}', name='get_current_stock'), type='function')
<class 'openai.types.chat.chat_completion_message_tool_call.ChatCompletionMessageToolCall'>
<class 'openai.types.chat.chat_completion_message_tool_call.Function'>


In [54]:
available_functions = {
    "get_current_stock": get_current_stock,
}

for tool_call in response_message.tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)

    function_response = function_to_call(
        compy_name= function_args['compy_name']

    )
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "function",
            "name": function_name,
            "content": json.dumps(function_response, ensure_ascii=False)
        }
    )
    
second_response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)
print(second_response.choices[0].message.content)

오늘 005930(삼성전자) 주가는 68,100원입니다.


In [53]:
print(messages)

[{'role': 'user', 'content': '오늘 005930 주가를 알려줘.'}, {'tool_call_id': 'call_TO4ARNyFp0GKrrlgCy9rQaxH', 'role': 'function', 'name': 'get_current_stock', 'content': '"\\"{\\\\\\"\\\\\\\\uc2dc\\\\\\\\uac00\\\\\\":{\\\\\\"1735776000000\\\\\\":52700,\\\\\\"1735862400000\\\\\\":52800,\\\\\\"1736121600000\\\\\\":54400,\\\\\\"1736208000000\\\\\\":56800,\\\\\\"1736294400000\\\\\\":54800,\\\\\\"1736380800000\\\\\\":57600,\\\\\\"1736467200000\\\\\\":56100,\\\\\\"1736726400000\\\\\\":54600,\\\\\\"1736812800000\\\\\\":54200,\\\\\\"1736899200000\\\\\\":54100,\\\\\\"1736985600000\\\\\\":54200,\\\\\\"1737072000000\\\\\\":53800,\\\\\\"1737331200000\\\\\\":53600,\\\\\\"1737417600000\\\\\\":53700,\\\\\\"1737504000000\\\\\\":53400,\\\\\\"1737590400000\\\\\\":53700,\\\\\\"1737676800000\\\\\\":53600,\\\\\\"1738281600000\\\\\\":52200,\\\\\\"1738540800000\\\\\\":51100,\\\\\\"1738627200000\\\\\\":51600,\\\\\\"1738713600000\\\\\\":53600,\\\\\\"1738800000000\\\\\\":53400,\\\\\\"1738886400000\\\\\\":53900,\\\\\\"1